# Tutorial: Creating Custom Aggregates with DynamicEventAggregator
This tutorial will guide you through using the `DynamicEventAggregator` to generate standard and custom event aggregates from match data.

## 📋 Prerequisites
- Python installed
- Access to the `skillcorner_analysis_lib` library
- Required libraries: `numpy`, `pandas`
```bash
pip install numpy pandas skillcorner
```

In [3]:
import numpy as np
import pandas as pd
from io import BytesIO
from DynamicEventsAggregator import DynamicEventAggregator
from skillcorner.client import SkillcornerClient
import os
import json

## ⚽ Step 2: Load Match Data
We will use the `GCSHelper` to load dynamic event data for a specific match.

In [5]:
# match_id = 1886347

# Instantiate the SkillCorner client
# username = "your_username"
# password = "your_password"
# client = SkillcornerClient(username=username, password=password)

# Load dynamic event data from SkillCorner API
# events_df = pd.read_csv(BytesIO(client.get_dynamic_events(MATCH_ID)))

# Load dynamic event data from Open Source
matches_json_path = os.path.join(os.getcwd(), "data/matches.json")

with open(matches_json_path, "r") as f:
    matches_json = json.load(f)

match_id = matches_json[0]["id"]

events_df = pd.read_csv(
    f"https://raw.githubusercontent.com/SkillCorner/opendata/master/data/matches/{match_id}/{match_id}_dynamic_events.csv"
)

C:\Users\g4a4b\AppData\Local\Temp\ipykernel_9812\777117629.py:19: DtypeWarning: Columns (75,77,184,264) have mixed types. Specify dtype option on import or set low_memory=False.
  events_df = pd.read_csv(


## 📊 Step 3: Generate Standard Aggregates
Create an instance of `DynamicEventAggregator` and generate predefined aggregates like off-ball runs, line-breaking passes, and defensive engagements.

In [6]:
# Initialize the aggregator
events_aggregator = DynamicEventAggregator(df=events_df)

# Off-ball runs
off_ball_runs = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="off_ball_runs"
)

# Line-breaking passes
line_breaking_passes = events_aggregator.generate_aggregates(
    group_by=["player_in_possession_id", "player_in_possession_name"],
    aggregate_type="line_breaking_passes",
)

# Defensive engagements
defensive_engagements = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="on_ball_engagements"
)

# Pressing
pressing = events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="pressing_engagements"
)

off_ball_runs.head()

c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\DynamicEventsAggregator.py:1555: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(
c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\DynamicEventsAggregator.py:1555: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(
c:\Users\g4a4b\OneDrive - University of Twente\Sport

,player_id,player_name,count_off_ball_runs,count_targeted_off_ball_runs,count_received_off_ball_runs,xthreat_off_ball_runs,xthreat_targeted_off_ball_runs,xthreat_received_off_ball_runs,xpass_completion_off_ball_runs,xpass_completion_targeted_off_ball_runs,...,count_dangerous_received_dropping_off_runs_in_create,count_difficult_dropping_off_runs_in_create,count_difficult_targeted_dropping_off_runs_in_create,count_difficult_received_dropping_off_runs_in_create,avg_speed_avg_dropping_off_runs_in_create,count_hsr_dropping_off_runs_in_create,count_sprint_dropping_off_runs_in_create,avg_distance_covered_dropping_off_runs_in_create,count_center_channel_dropping_off_runs_in_create,count_wide_channel_dropping_off_runs_in_create
0,4322,H. Sakai,18.0,6.0,5.0,0.0928,0.0176,0.0091,14.1037,4.8099,...,0.0,0.0,0.0,0.0,18.375,0.0,0.0,8.98,0.0,2.0
1,11117,Z. Machach,25.0,12.0,8.0,0.8244,0.3175,0.0857,16.1596,8.0390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11885,D. Arzani,41.0,16.0,10.0,0.9581,0.1539,0.0989,30.3404,12.2003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14736,L. Verstraete,17.0,5.0,3.0,0.0838,0.0543,0.0369,13.2469,3.7362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23418,F. Gallegos,24.0,12.0,11.0,0.1858,0.1045,0.1033,20.9220,10.1885,...,0.0,0.0,0.0,0.0,16.220,0.0,0.0,7.28,0.0,0.0


### Note there are now 821 columns !

## 🛠️ Step 4: Create Custom Aggregates
You can define your own contexts and metrics to tailor the analysis.

### Define Custom Contexts

In [7]:
contexts = {
    "custom": {
        "all_possessions": (
            (events_df["event_type"] == "player_possession")
            & (events_df["team_in_possession_phase_type"] == "finish")
            & (events_df["separation_start"] >= 5)
        )
    }
}

### Define Custom Metrics

In [8]:
metric = {
    "custom": {
        "count": lambda x: len(x),
        "avg_duration": lambda x: x["duration"].mean(),
        "avg_distance_covered": lambda x: x["distance_covered"].mean(),
    }
}

### Generate Custom Aggregates

In [9]:
# Initialize aggregator with custom contexts and metrics
custom_events_aggregator = DynamicEventAggregator(
    df=events_df, custom_context_groups=contexts, custom_metric_groups=metric
)

# Generate custom aggregates
custom_aggregates = custom_events_aggregator.generate_aggregates(
    group_by=["player_id", "player_name"], aggregate_type="custom"
)

# Display the results
custom_aggregates.head()

c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\DynamicEventsAggregator.py:1555: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,player_id,player_name,count_all_possessions,avg_duration_all_possessions,avg_distance_covered_all_possessions
0,4322,H. Sakai,6.0,1.55,4.5700
1,11117,Z. Machach,4.0,2.80,9.5925
2,11885,D. Arzani,4.0,3.95,12.6775
3,14736,L. Verstraete,1.0,0.00,0.0000
4,23418,F. Gallegos,2.0,2.20,8.6650
